In [83]:
import requests
import psycopg2
from psycopg2 import sql

url = 'https://api.exchangerate.host/timeseries?start_date=2023-01-01&end_date=2023-01-31&base=BTC&symbols=RUB'
response = requests.get(url)
data = response.json()

lst = []
for i in data['rates']:
    for j in data['rates'][i]:
        lst.append((i, data['rates'][i][j]))

conn = psycopg2.connect(dbname='testdb', user='postgres', 
                        password='postgres', host='localhost')

with conn.cursor() as cursor:
    conn.autocommit = True
    insert = sql.SQL('INSERT INTO exchange_rate (rate_dt, rate_amt) VALUES {}').format(sql.SQL(',').join(map(sql.Literal, lst)))
    cursor.execute(insert)

    res = []

    cursor.execute('SELECT * FROM exchange_rate')
    max_rate_dt = ''
    min_rate_dt = ''
    max_rate_amt = 0
    min_rate_amt = 0
    avg_rate_amt = 0
    last_date_rate_amt = 0
    i = 0
    for row in cursor:
        i += 1
        avg_rate_amt = avg_rate_amt + row[2]
        last_date_rate_amt = row[2]
        if row[2] > max_rate_amt:
            max_rate_dt = row[1]
            max_rate_amt = row[2]
        elif min_rate_amt == 0 or row[2] < min_rate_amt:
            min_rate_amt = row[2]
            min_rate_dt = row[1]
    avg_rate_amt = avg_rate_amt / i

    res.append((data['start_date'], data['end_date'], data['base'], max_rate_dt, min_rate_dt, max_rate_amt, min_rate_amt, avg_rate_amt, last_date_rate_amt))

    insert = sql.SQL('INSERT INTO rates_by_month (start_date, end_date, currency, max_rate_dt, min_rate_dt, max_rate_amt, min_rate_amt, avg_rate_amt, last_date_rate_amt) VALUES {}').format(sql.SQL(',').join(map(sql.Literal, res)))
    cursor.execute(insert)

2023-01-01
